# Project: Build a Multi-user Conversational Product Recommendation Agent

![](https://i.imgur.com/7ZHpO6U.png)


___Created By: Dipanjan (DJ)___

## Install OpenAI and LangChain dependencies


In [0]:
!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11
!pip install gdown
!pip install rich

In [0]:
dbutils.library.restartPython()

## Load OpenAI API Credentials

Here we load it from get password function

## Enter API Tokens

In [0]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

In [0]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

## Get Dataset of Products

In [0]:
# download it manually from https://drive.google.com/file/d/1tAwsv97fICL74uJH9fDlxNEZ_YFFJS3W/view?usp=sharing
# or use gdown as follows to download it automatically
!gdown 1tAwsv97fICL74uJH9fDlxNEZ_YFFJS3W

In [0]:
import pandas as pd

df = pd.read_csv('./Ecommerce_Product_List.csv')
df.head()

In [0]:
df.info()

In [0]:
df['Category'].unique()

In [0]:
df['Rating'].unique()

## Product Recommender Agent Workflow

We will:

- Build a Pandas Code Tool Executor to filter products based on category, rating, price
- Build a LLM Recommender Chain to filter products based on natural language descriptions using an LLM
- Build a query rephraser LLM Chain to combine multiple queries in a conversation to generate better queries
- Combine all of these into a single chain
- Add conversational memory to this system

![](https://i.imgur.com/qJIsErH.png)

## Text 2 Pandas Code Tool Executor Chain

In [0]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import chain
from langchain.schema.runnable import RunnablePassthrough
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

@chain
def pandas_code_tool_executor(query):
    result_df = eval(query)
    if result_df.empty:
        return df.to_markdown()
    else:
        return result_df.to_markdown()

FILTER_PROMPT = """Given the following schema of a dataframe table,
            your task is to figure out the best pandas query to
            filter the dataframe based on the user query which
            will be in natural language.

            The schema is as follows:

            #   Column        Non-Null Count  Dtype
            ---  ------        --------------  -----
            0   Product_ID    30 non-null     object
            1   Product_Name  30 non-null     object
            2   Category      30 non-null     object
            3   Price_USD     30 non-null     int64
            4   Rating        30 non-null     float64
            5   Description   30 non-null     object

            Category has values: ['Laptop', 'Tablet', 'Smartphone',
                                  'Smartwatch', 'Camera',
                                  'Headphones', 'Mouse', 'Keyboard',
                                  'Monitor', 'Charger']

            Rating ranges from 1 - 5 in floats

            You will try to figure out the pandas query focusing
            only on Category, Price_USD and Rating if the user mentions
            anything about these in their natural language query.
            Do not make up column names, only use the above.
            If not the pandas query should just return the full dataframe.
            Remember the dataframe name is df.

            Just return only the pandas query and nothing else.
            Do not return the results as markdown, just return the query

            User Query: {user_query}
            Pandas Query:
        """

filter_prompt_template = ChatPromptTemplate.from_template(FILTER_PROMPT)

data_filter_chain = (
         filter_prompt_template
           |
         chatgpt
           |
         StrOutputParser()
           |
         pandas_code_tool_executor
)

product_table = data_filter_chain.invoke({"user_query": """looking for a tablet with > 10 inch display
                                                           and at least 64GB storage"""})
print(product_table)

## Product Description LLM Recommender Chain

In [0]:
RECOMMEND_PROMPT = """Act as an expert retail product advisor
                      Given the following table of products,
                      focus on the product attributes and description in the table
                      and based on the user query below do the following

                      - Recommend the most appropriate products based on the query
                      - Recommedation should have product name, price,  rating, description
                      - Also add a brief on why you recommend the product
                      - Do not make up products or recommend products not in the table
                      - If some specifications do not match focus on the ones which match and recommend
                      - If nothing matches recommend 5 random products from the table
                      - Do not generate anything else except the fields mentioned above

                    In case the user query is just a generic query or greeting
                    respond to them appropriately without recommending any products

                    Product Table:
                    {product_table}

                    User Query:
                    {user_query}

                    Recommendation:
                    """

recommend_prompt_template = ChatPromptTemplate.from_template(RECOMMEND_PROMPT)

recommend_chain = (
         recommend_prompt_template
           |
         chatgpt
           |
         StrOutputParser()
)

response = recommend_chain.invoke({"user_query": """looking for a tablet with greater than 10 inch display
                                                           and at least 64GB storage""",
                                   "product_table": product_table})
print(response)

In [0]:
combined_chain = (
         {
             'user_query' : itemgetter('user_query'),
             'product_table' : data_filter_chain
         }
           |
         recommend_chain
)

In [0]:
response = combined_chain.invoke({"user_query": """looking for a cheap laptop
                                                      in the range of 500 - 1000
                                                """})
print(response)

## Multi-user Window-based Conversation Chains with persistence

The beauty of `SQLChatMessageHistory` is that we can store separate conversation histories per user or session which is often the need for real-world chatbots which will be accessed by many users at the same time. Instead of in-memory we can store it in a SQL database which can be used to store a lot of conversations.

We use a `get_session_history` function which is expected to take in a `session_id` and return a Message History object. Everything is stored in a SQL database. This `session_id` is used to distinguish between separate conversations, and should be passed in as part of the config when calling the new chain

We also use a `memory_buffer_window` function to only use the top-K last historical conversations before sending it to the LLM, basically our own implementation of `ConversationBufferWindowMemory`

In [0]:
# removes the memory database file - usually not needed
# you can run this only when you want to remove all conversation histories
!rm memory.db

## Historical Conversation Query Rephraser Chain

In [0]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# used to retrieve conversation history from database
# based on a specific user or session ID
def get_session_history_db(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")


SYS_PROMPT = """You are a retail product expert.
                Carefully analyze the following conversation history
                and the current user query.
                Refer to the history and rephrase the current user query
                into a standalone query which can be used without the history
                for making search queries.
                Rephrase only if needed.
                Just return the query and do not answer it.
            """

# prompt to load in history and current input from the user
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        MessagesPlaceholder(variable_name="history"),
        ("human", """Current User Query:
                     {human_input}
                  """),
    ]
)

# create a memory buffer window function to return the last K conversations
def memory_buffer_window(messages, k=10): # 10 here means retrieve only last 2*10 user-AI conversations
    return messages[-(2*k):]

# create a basic LLM Chain which only sends the last K conversations per user
rephrase_query_chain = (
    RunnablePassthrough.assign(history=lambda x: memory_buffer_window(x["history"]))
      |
    prompt_template
      |
    chatgpt
      |
    StrOutputParser()
)

## Combining All Chains Together

In [0]:
combined_chain = (
         {
             'human_input' : itemgetter('human_input'),
             'history' : itemgetter('history')
         }
           |
        {
            'user_query': rephrase_query_chain
        }
           |
        RunnablePassthrough.assign(product_table=data_filter_chain)
            |
        recommend_chain
)

## Wrapping it into a Multi-user Conversation Chain with Memory

In [0]:
from rich.console import Console
from rich.markdown import Markdown

# create a conversation chain which can load memory based on specific user or session id
conv_chain = RunnableWithMessageHistory(
    combined_chain,
    get_session_history_db,
    input_messages_key="human_input",
    history_messages_key="history",
)

# create a utility function to take in current user input prompt and their session ID
# streams result live back to the user from the LLM
def chat_with_llm(prompt: str, session_id: str):
    response = conv_chain.invoke({"human_input": prompt},
                                 {'configurable': { 'session_id': session_id}})
    console = Console()
    console.print(Markdown(response))


## Test Product Recommender Agent

Test conversation chain for user 1

In [0]:
user_id = 'jim001'
prompt = "looking for a tablet"
chat_with_llm(prompt, user_id)

In [0]:
prompt = "want one which has display larger than 10 inches"
chat_with_llm(prompt, user_id)

In [0]:
prompt = "need at least 128GB disk space"
chat_with_llm(prompt, user_id)

Now test conversation chain for user 2

In [0]:
user_id = 'bond007'
prompt = "I want a laptop with a high rating"
chat_with_llm(prompt, user_id)

In [0]:
prompt = "want atleast 16GB memory"
chat_with_llm(prompt, user_id)